In [1]:
import sys
import numpy as np
import tensorflow as tf
import os
from tensorflow.contrib.lookup import MutableHashTable
from tensor2tensor.layers import common_layers

from tensor2tensor.models import transformer
sys.path.append('/workspace/MT/tensor2tensor/tensor2tensor/models/')
sys.path.append('/workspace/MT/tensor2tensor/tensor2tensor/utils/')

from transformer_test import TransformerTest
from transformer_cache import TransformerCache
from tensor2tensor.data_generators import problem_hparams
from tensor2tensor.data_generators import problem

sys.path.append('/workspace/MT/t2t_data_generators/')

from generator import ShadENRUOpusProblem

In [2]:
%env CUDA_VISIBLE_DEVICES=4

env: CUDA_VISIBLE_DEVICES=4


In [3]:
shad_problem = ShadENRUOpusProblem()

data = shad_problem.generate_encoded_samples(
    '/workspace/MT/shad_nlp18_contextNMT/data_4prev/',
    '/workspace/MT/shad_nlp18_contextNMT/data_4prev/',
    problem.DatasetSplit.TEST
)

INFO:tensorflow:Found vocab file: /workspace/MT/shad_nlp18_contextNMT/data_4prev/vocab_enru.src


[2018-04-26 10:43:31,691] Found vocab file: /workspace/MT/shad_nlp18_contextNMT/data_4prev/vocab_enru.src


INFO:tensorflow:Found vocab file: /workspace/MT/shad_nlp18_contextNMT/data_4prev/vocab_enru.dst


[2018-04-26 10:43:31,772] Found vocab file: /workspace/MT/shad_nlp18_contextNMT/data_4prev/vocab_enru.dst


In [4]:
has_input = True
mode=tf.estimator.ModeKeys.EVAL

hparams = transformer.transformer_base_single_gpu()
hparams.data_dir =  '/workspace/MT/shad_nlp18_contextNMT/data_4prev//'
p_hparams = shad_problem.get_hparams(hparams)

In [5]:
hparams.batch_size = 1
hparams.use_cache = False

## Initializing the model

In [6]:
model = TransformerCache(hparams, mode, p_hparams)

INFO:tensorflow:Unsetting shared_embedding_and_softmax_weights.


[2018-04-26 10:43:32,191] Unsetting shared_embedding_and_softmax_weights.


INFO:tensorflow:Setting T2TModel mode to 'eval'


[2018-04-26 10:43:32,219] Setting T2TModel mode to 'eval'


INFO:tensorflow:Setting hparams.relu_dropout to 0.0


...

[2018-04-26 10:43:32,223] Setting hparams.dropout to 0.0


INFO:tensorflow:Setting hparams.symbol_dropout to 0.0


[2018-04-26 10:43:32,224] Setting hparams.symbol_dropout to 0.0


INFO:tensorflow:Setting hparams.layer_prepostprocess_dropout to 0.0


[2018-04-26 10:43:32,226] Setting hparams.layer_prepostprocess_dropout to 0.0


In [7]:
features_true = {
    "inputs": tf.placeholder(shape=[None, None, 1, 1], dtype=tf.int32, name="inputs"),
    "targets": tf.placeholder(shape=[None, None, 1, 1], dtype=tf.int32, name="inputs"),
    "target_space_id": tf.constant(1, dtype=tf.int32)
}

In [8]:
data = shad_problem.generate_encoded_samples(
    '/workspace/MT/shad_nlp18_contextNMT/data_4prev/',
    '/workspace/MT/shad_nlp18_contextNMT/data_4prev/',
    problem.DatasetSplit.TEST
)

INFO:tensorflow:Found vocab file: /workspace/MT/shad_nlp18_contextNMT/data_4prev/vocab_enru.src


[2018-04-26 10:43:32,297] Found vocab file: /workspace/MT/shad_nlp18_contextNMT/data_4prev/vocab_enru.src


INFO:tensorflow:Found vocab file: /workspace/MT/shad_nlp18_contextNMT/data_4prev/vocab_enru.dst


[2018-04-26 10:43:32,433] Found vocab file: /workspace/MT/shad_nlp18_contextNMT/data_4prev/vocab_enru.dst


In [9]:
decode_length = tf.placeholder(dtype=tf.int32)

In [10]:
fast_result = model._beam_decode(features_true, decode_length, beam_size=2, top_beams=1, alpha=1.0)["outputs"]

Tensor("transformer_cache/while/Identity:0", shape=(), dtype=int32)
KEK


## Loading the weights

In [11]:
save_dict = {"/".join(["transformer"] + var.name[:-2].split('/')[1:]) : var for var in tf.global_variables()[5:]}

In [12]:
saver = tf.train.Saver(save_dict)
ckpt = tf.train.get_checkpoint_state('/workspace/MT/train/transformer/')

In [13]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [14]:
if ckpt and ckpt.model_checkpoint_path:
    saver.restore(sess, ckpt.model_checkpoint_path)
else:
    print("...no checkpoint found...")

INFO:tensorflow:Restoring parameters from /workspace/MT/train/transformer/model.ckpt-250000


[2018-04-26 10:43:44,544] Restoring parameters from /workspace/MT/train/transformer/model.ckpt-250000


In [15]:
new_saver = tf.train.Saver(tf.global_variables()[2:4])
new_ckpt = tf.train.get_checkpoint_state('/workspace/MT/train_cache/')

In [16]:
from tensorflow.python.tools.inspect_checkpoint import print_tensors_in_checkpoint_file

In [17]:
print_tensors_in_checkpoint_file(file_name=new_ckpt.model_checkpoint_path, tensor_name='',all_tensors=True)

tensor_name:  sentence_level_cache/m_weight
[[-0.0662327  -0.03821926 -0.01203594 ...,  0.01267674 -0.06697603
  -0.00703066]
 [ 0.06985063 -0.06531873  0.053579   ..., -0.04061517  0.04970364
  -0.03511227]
 [-0.01632818  0.0105549   0.00983872 ..., -0.05448194 -0.00812256
   0.0255211 ]
 ..., 
 [-0.03303513  0.02034533 -0.01472183 ..., -0.03932308 -0.0053694
  -0.03564095]
 [ 0.04379536 -0.07470047  0.03121056 ...,  0.02519272  0.05219584
  -0.03582729]
 [ 0.01609153 -0.00867114  0.02021801 ..., -0.041036    0.02171655
  -0.03784646]]
tensor_name:  sentence_level_cache/s_weight
[[-0.30897236 -0.0865991  -0.04653369 ..., -0.09849302 -0.08101944
  -0.0917303 ]
 [-0.03588444 -0.11626038 -0.04050927 ..., -0.03733981 -0.03060537
  -0.03144538]
 [-0.04808292 -0.03741568 -0.25722969 ..., -0.04486312 -0.06077321
  -0.0150316 ]
 ..., 
 [-0.03111542 -0.04330374 -0.04637462 ...,  0.07261162 -0.03328684
  -0.05452815]
 [-0.05466266 -0.06009268 -0.04546668 ..., -0.06550934 -0.01551719
  -0.034632

In [18]:
if new_ckpt and new_ckpt.model_checkpoint_path:
    new_saver.restore(sess, new_ckpt.model_checkpoint_path)
else:
    print("...no checkpoint found...")

INFO:tensorflow:Restoring parameters from /workspace/MT/train_cache/transformer_cache


[2018-04-26 10:43:45,105] Restoring parameters from /workspace/MT/train_cache/transformer_cache


In [19]:
encoders = shad_problem.feature_encoders('/workspace/MT/shad_nlp18_contextNMT/data_4prev/')

## Predicting

In [20]:
from itertools import zip_longest

def box_hpaulj(LoL):
    return np.array(list(zip_longest(*LoL, fillvalue=0)), dtype=np.int32).T

def batch_iterator(data, batch_size, context_size, n_steps=1000):
    
    for k in range(n_steps):
        
        batch_list_inp = [[] for _ in range(context_size)]
        batch_list_targ = [[] for _ in range(context_size)]

        for batch_i in range(batch_size):
            for context_i, p in enumerate(data):
                batch_list_inp[context_i].append(p['inputs'])
                batch_list_targ[context_i].append(p['targets'])
                if context_i == context_size - 1:
                    break

        for b_inp, b_out in zip(batch_list_inp, batch_list_targ):
            yield {'inputs' : box_hpaulj(b_inp), 'targets' : box_hpaulj(b_out)}

In [21]:
data = shad_problem.generate_encoded_samples(
    '/workspace/MT/shad_nlp18_contextNMT/data_4prev/',
    '/workspace/MT/shad_nlp18_contextNMT/data_4prev/',
    problem.DatasetSplit.TEST,
)

INFO:tensorflow:Found vocab file: /workspace/MT/shad_nlp18_contextNMT/data_4prev/vocab_enru.src


[2018-04-26 10:43:45,794] Found vocab file: /workspace/MT/shad_nlp18_contextNMT/data_4prev/vocab_enru.src


INFO:tensorflow:Found vocab file: /workspace/MT/shad_nlp18_contextNMT/data_4prev/vocab_enru.dst


[2018-04-26 10:43:45,922] Found vocab file: /workspace/MT/shad_nlp18_contextNMT/data_4prev/vocab_enru.dst


In [22]:
hparams.batch_size

1

In [23]:
counter = 0
flusher = 0

In [24]:
for it in batch_iterator(data, hparams.batch_size, 5, n_steps=10000):
    
    inputs = it['inputs'].reshape((it['inputs'].shape[0], it['inputs'].shape[1], 1, 1))
    targets = it['targets'].reshape((it['targets'].shape[0], it['targets'].shape[1], 1, 1))
    break
    kek = sess.run([fast_result], feed_dict={
        features_true['inputs'] : inputs,
        features_true['targets'] : targets,
        decode_length : 25
    })
    
    with open('/workspace/MT/transformer_cache_pred_ru.dst', 'a', encoding='utf8') as file_pred:
        with open('/workspace/MT/transformer_cache_true_ru.dst', 'a', encoding='utf8') as file_true:
            for seq_i in range(hparams.batch_size):
                pred = encoders['targets'].decode(kek[0][seq_i][:-1])
                true = encoders['targets'].decode(targets[seq_i][:-1].squeeze(1).squeeze(1))
                print(''.join(pred.split('<EOS>')[0]), file=file_pred)
                print(''.join(true.split('<EOS>')[0]), file=file_true)
                
    counter += 1
    flusher += 1
    
    if flusher == 5:
        sess.run(model.sentence_cache.Flush())
        flusher = 0
    
    if counter == 60:
        break

In [24]:
inp = [
    'How do you choose the matrix columns?',
    'You just choose the columns at random',
]

In [25]:
fake_inp = [encoders['inputs'].encode(t) for t in inp]

In [26]:
fake_inp

[[122, 39, 6, 1263, 8, 4670, 15341, 22026, 21995],
 [16, 44, 1263, 8, 15341, 63, 4573]]

In [27]:
box_hpaulj(fake_inp[0:1])

array([[  122,    39,     6,  1263,     8,  4670, 15341, 22026, 21995]], dtype=int32)

In [28]:
sess.run(model.sentence_cache.Flush())

0.0

In [29]:
kek = sess.run([fast_result], feed_dict={
        features_true['inputs'] : box_hpaulj(fake_inp[0:1]).reshape(1, -1, 1, 1),
        features_true['targets'] : box_hpaulj(fake_inp[0:1]).reshape(1, -1, 1, 1),
        decode_length : 25
})

In [30]:
encoders['targets'].decode(kek[0][0][:-1])

'Как вы можете выбирать климатрицы матрицы?<EOS><pad><pad><pad><pad><pad>'

In [31]:
kek = sess.run([fast_result], feed_dict={
        features_true['inputs'] : box_hpaulj(fake_inp[1:2]).reshape(1, -1, 1, 1),
        features_true['targets'] : box_hpaulj(fake_inp[0:1]).reshape(1, -1, 1, 1),
        decode_length : 25
})

In [32]:
encoders['targets'].decode(kek[0][0][:-1])

'Вы просто выбираете колонны из случайной колонки , выбрав случайной колонны , вы просто выбираете Котхарактеры .<EOS><pad>'

In [28]:
sess.run(model.sentence_cache.Flush())

0.0

In [29]:
kek = sess.run([fast_result], feed_dict={
        features_true['inputs'] : box_hpaulj(fake_inp[0:1]).reshape(1, -1, 1, 1),
        features_true['targets'] : box_hpaulj(fake_inp[0:1]).reshape(1, -1, 1, 1),
        decode_length : 25
})

In [32]:
encoders['targets'].decode(kek[0][0][:-1])

'Как вы выбираете матрицу матрицу спутне?<EOS>'

In [33]:
kek = sess.run([fast_result], feed_dict={
        features_true['inputs'] : box_hpaulj(fake_inp[1:2]).reshape(1, -1, 1, 1),
        features_true['targets'] : box_hpaulj(fake_inp[0:1]).reshape(1, -1, 1, 1),
        decode_length : 25
})

In [34]:
encoders['targets'].decode(kek[0][0][:-1])

'Вы просто выбирайте слова , что в теле по случайносттратить округа по ходит по ходит по обойти обойти округих обойти обойти обойти обойти обойти обойти обойти обойти'

In [ ]:
encoders['inputs'].decode(targets[5][:-1].squeeze(1).squeeze(1))

In [ ]:
encoders['targets'].decode(kek[1][23][:-1])

In [ ]:
encoders['targets'].decode(targets[i][:-1].squeeze(1).squeeze(1))

In [ ]:
counter = 0
flusher = 0
with open('/workspace/MT/transformer_cache_test_ru.dst', 'w', encoding='utf8') as file:
    for it in data:
        inputs = np.array(it['inputs']).reshape((1, len(it['inputs']), 1, 1))
        targets = np.array(it['targets']).reshape((1, len(it['targets']), 1, 1))
        kek = sess.run([fast_result], feed_dict={
            features_true['inputs'] : inputs,
            features_true['targets'] : targets,
            decode_length : len(targets)
        })
        string = encoders['targets'].decode(np.squeeze(kek[0])[:-1])
        print(string, file=file)
        counter += 1
        flusher += 1
        if flusher == 5:
            model.sentence_cache.Flush()
            flusher = 0
        if counter == 5000:
            break

In [ ]:
counter = 0
with open('/workspace/MT/shad_nlp18_contextNMT/data_fused/en_test.src', 'r', encoding='utf8') as file:
    with open('/workspace/MT/shad_nlp18_contextNMT/data_fused/en_test_short.src', 'w', encoding='utf8') as wfile:
        for line in file:
            print(line, file=wfile)
            counter += 1
            if counter == 5000:
                break

In [ ]:
counter = 0
with open('/workspace/MT/shad_nlp18_contextNMT/data_fused/ru_test.dst', 'r', encoding='utf8') as file:
    with open('/workspace/MT/shad_nlp18_contextNMT/data_fused/ru_test_short.dst', 'w', encoding='utf8') as wfile:
        for line in file:
            print(line, end='', file=wfile)
            counter += 1
            if counter == 5000:
                break